# Tugas 2

In [21]:
from pathlib import Path
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
# Image directories
train_dir = "images/training/"
test_dir = "images/test/"

In [3]:
def load_dataset(img_dir):
    p = Path(img_dir)
    dirs = p.glob('*')

    img_list = []

    for dir in dirs:
        label = str(dir).split('/')[-1]
        for file in dir.glob('*.jpg'):
            img = mpimg.imread(file)

            if not img is None:
                img_list.append((img, label))

    return img_list

In [4]:
train_img = load_dataset(train_dir)
test_img = load_dataset(test_dir)

In [5]:
pick_random = np.random.randint(0, len(train_img))

print(f'Image {pick_random}')
print(train_img[pick_random][0].shape)

Image 53
(700, 1280, 3)


In [6]:
def random_img_viz(img_list):
    rand_num = np.random.randint(0, len(img_list))

    img = img_list[rand_num][0]
    label = img_list[rand_num][1]
    label_str = 'day' if label == 1 else 'night'

    plt.imshow(img)
    print(f'Shape\t: {img.shape}')
    print(f'Label\t: {label}')

In [7]:
def standarized_input(image):
    # resize to w: 1100, h:600
    std_img = cv2.resize(image, (1100,600))

    return std_img

In [8]:
def label_encoder(label):
    # Extract the label by splitting the path and taking the last part
    label = label.split('\\')[-1].strip().lower()  # for Windows-style paths
    num_val = 0
    if label == 'day':
        num_val = 1

    return num_val

In [9]:
def preprocess(img_list):
    std_img_list = []

    for item in img_list:
        image = item[0]
        label = item[1]

        std_img = standarized_input(image)

        img_label = label_encoder(label)

        std_img_list.append((std_img, img_label))

    return std_img_list

In [10]:
train_std_img_list = preprocess(train_img)
test_std_img_list = preprocess(test_img)

In [11]:
pick_random = np.random.randint(0, len(train_std_img_list))

print(f'Image {pick_random}')
print(train_std_img_list[pick_random][0].shape)

Image 30
(600, 1100, 3)


In [23]:
def extract_grayscale_histogram(image, bins=256):
    # Ubah gambar menjadi grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Hitung histogram grayscale
    hist = cv2.calcHist([gray_image], [0], None, [bins], [0, 256])
    
    # Normalisasi histogram
    hist = cv2.normalize(hist, hist).flatten()
    
    return hist

In [24]:
def extract_histogram_feature(img_list):
    hist_features = []
    labels = []

    for img in img_list:
        hist = extract_grayscale_histogram(img[0])
        img_label = img[1]

        hist_features.append(hist)
        labels.append(img_label)

    df = pd.DataFrame(hist_features)
    df['LABELS'] = labels

    return df

In [26]:
train_hist_img = extract_histogram_feature(train_std_img_list)
print(f'Shape: {train_hist_img.shape}')
train_hist_img.head()

Shape: (240, 257)


,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,LABELS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000031,0.000063,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000356,0.000386,0.000234,0.000112,0.000152,0.000122,0.000061,0.000000,0.000000,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000424,0.000338,0.000405,0.000367,0.000453,0.000367,0.000357,0.000376,0.000039,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1


In [28]:
test_hist_img = extract_histogram_feature(test_std_img_list)
print(f'Shape: {test_hist_img.shape}')
test_hist_img.head()

Shape: (160, 257)


,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,LABELS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000298,0.000318,0.000277,0.000236,0.000390,0.000359,0.000359,0.000205,0.000000,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.027545,0.032145,0.036408,0.047815,0.043496,0.037643,0.038989,0.031191,0.013034,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000994,0.001203,0.001362,0.001143,0.001173,0.000845,0.000805,0.000378,0.000080,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000194,0.000068,0.000068,0.000058,0.000000,0.000029,0.000000,0.000000,0.000000,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000048,0.000010,0.000019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1


In [29]:
x_train = train_hist_img.iloc[:, :-1].values
y_train = train_hist_img.iloc[:, -1].values
x_test = test_hist_img.iloc[:, :-1].values
y_test = test_hist_img.iloc[:, -1].values

In [30]:
model_hist = SVC(kernel='linear')
model_hist.fit(x_train, y_train)

SVC(kernel='linear')

In [31]:
y_train_pred_hist = model_hist.predict(x_train)
y_test_pred_hist = model_hist.predict(x_test)

In [32]:
acc_train_hist = accuracy_score(y_train, y_train_pred_hist)
acc_test_hist = accuracy_score(y_test, y_test_pred_hist)


print(f'Accuracy on train Histogram: {acc_train_hist}')
print(f'Accuracy on test Histogram: {acc_test_hist}')

Accuracy on train Histogram: 0.9708333333333333
Accuracy on test Histogram: 0.9
